# Wastewater SARS-CoV-2 Data

After sequencing, we are left with the following data:

| Read | Sequence |
|------|----------|
| 1    | ATCA     |
| 2    | TCAC     |
| 3    | CACTA    |
| 4    | TCAATA   |
| 5    | TCTC     |
| 6    | TCTC     |

Alignment is the process of matching these reads up to a reference, which in this toy example results in the following:

| Read      | 1 | 2 | 3 | 4 | 5 | 6 | 7 | ... |
|-----------|---|---|---|---|---|---|---|-----|
| Reference | A | T | C | A | C | T | A | ... |
| 1         | A | T | C | A | - | - | - | ... |
| 2         | - | T | C | A | C | - | - | ... |
| 3         | - | - | C | A | C | T | A | ... |
| 4         | - | T | C | A | A | T | A | ... |
| 5         | - | T | C | T | C | - | - | ... |
| 6         | - | T | C | T | C | - | - | ... |
| Depth     | 1 | 5 | 6 | 6 | 5 | 2 | 2 | ... | 

A couple interesting things:

- This example is 1-indexed. Always good to know when you're working with genome positions.
- The read depth is the total number of reads that aligned with a given position on the genome.
- Read 4 has a mutation at position 5, where C changed to A.
    - This mutation would be recorded by gromstole as `C5A`.
    - There were 5 reads that were aligned to this position, so we have 1 mutation out of 5 reads.
- Similarly, mutation `A4T` has a count of 2 and a coverage of 6, leading to a frequency of 0.33.
- Paired-end reads are a little trickier to deal with and are different depending on who you ask.

In the `gromstole` pipeline, these Single Nucleotide Variations will not include the reference base and the two mutations shown are simply `~5A` and `~4T`.

- Insertions and deletions can also occur - a deletion of 3 bases at position 5 would be labelled `-5.3` and an insertion of 3 bases at position 5 would be `+5.3`.

# Amino acid changes

With SARS-CoV-2, we know the positions of the amino acids (special groups of 3 bases), and the genome is split up into coding regions (e.g. the "spike protein").


In [1]:
provoc:::parse_mutations(c("~6037A", "~6037G"))

[1] "aa:orf1a:S1924R" "aa:orf1a:S1924R"

The output indicates that a mutation to A (from C) at location 6037 is an amino acid change from S to R at amino acid number 1924 in Open Reading Frame 1A. If the C had mutated to a G instead, we would get the same amino acid change. 